In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25336
2,Huelva,Confirmados PDIA 14 días,545
3,Huelva,Tasa PDIA 14 días,"106,20262291248514"
4,Huelva,Confirmados PDIA 7 días,300
5,Huelva,Total Confirmados,25536
6,Huelva,Curados,20291
7,Huelva,Fallecidos,349


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25336.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6664.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4495 personas en los últimos 7 días 

Un positivo PCR cada 1970 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25336.0,300.0,545.0,513170.0,58.460159,106.202623,107.0
Condado-Campiña,7275.0,183.0,316.0,156231.0,117.134244,202.264595,71.0
Huelva-Costa,15326.0,102.0,201.0,289548.0,35.227320,69.418542,31.0
Almonte,976.0,67.0,98.0,24507.0,273.391276,399.885747,21.0
Rociana del Condado,355.0,31.0,42.0,7939.0,390.477390,529.033883,14.0
Moguer,873.0,16.0,39.0,21867.0,73.169616,178.350940,11.0
Huelva (capital),6664.0,32.0,73.0,143837.0,22.247405,50.751893,8.0
Bollullos Par del Condado,652.0,19.0,31.0,14387.0,132.063669,215.472301,6.0
Lepe,2016.0,8.0,16.0,27880.0,28.694405,57.388809,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Alosno,148.0,20.0,26.0,3933.0,508.517671,661.072972,3.0
Rociana del Condado,355.0,31.0,42.0,7939.0,390.477390,529.033883,14.0
San Bartolomé de la Torre,220.0,10.0,17.0,3761.0,265.886732,452.007445,4.0
Almonte,976.0,67.0,98.0,24507.0,273.391276,399.885747,21.0
Villalba del Alcor,460.0,3.0,13.0,3366.0,89.126560,386.215092,1.0
Puebla de Guzmán,110.0,2.0,8.0,3092.0,64.683053,258.732212,0.0
Niebla,167.0,8.0,10.0,4116.0,194.363460,242.954325,2.0
Bollullos Par del Condado,652.0,19.0,31.0,14387.0,132.063669,215.472301,6.0
Condado-Campiña,7275.0,183.0,316.0,156231.0,117.134244,202.264595,71.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,460.0,3.0,13.0,3366.0,89.126560,386.215092,1.0,0.230769
Aljaraque,929.0,1.0,4.0,21474.0,4.656794,18.627177,0.0,0.250000
Puebla de Guzmán,110.0,2.0,8.0,3092.0,64.683053,258.732212,0.0,0.250000
Villarrasa,133.0,1.0,4.0,2211.0,45.228403,180.913614,0.0,0.250000
Isla Cristina,2008.0,4.0,14.0,21393.0,18.697705,65.441967,2.0,0.285714
Bonares,181.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Gibraleón,536.0,8.0,23.0,12737.0,62.809139,180.576274,1.0,0.347826
Valverde del Camino,427.0,4.0,10.0,12750.0,31.372549,78.431373,1.0,0.400000
Moguer,873.0,16.0,39.0,21867.0,73.169616,178.350940,11.0,0.410256
